# Doppler imaging

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In this notebook we will show how to use ``starry`` to model Doppler imaging datasets.

In [ ]:
import starry
import numpy as np
import matplotlib.pyplot as plt

## Star with a grey spot

Let's instantiate a ``DopplerMap``. In addition to the spherical harmonic degree, we need to provide the number of epochs we're planning on modeling (``nt``).

In [ ]:
map = starry.DopplerMap(ydeg=15, nt=20)

Specifically, we're modeling a degree `15` spherical harmonic map with `20` epochs (i.e., the number of spectra we've observed and would like to model).

Let's specify two more properties relevant to Doppler imaging: the stellar inclination and the equatorial velocity.

In [ ]:
map.inc = 60
map.veq = 30000

By default, these are in degrees and meters per second, respectively.

The final thing we must do is specify what the surface of the star looks like, both spatially *and* spectrally. The spatial map can be specified by loading a latitude-longitude map, either as an image file name or an `ndarray`:

In [ ]:
map.load(images="spot")

The map we loaded above is the image located at ``starry/img/spot.png``, which looks like this on a rectangular latitude-longitude grid:

In [ ]:
import pathlib

file = pathlib.Path().absolute().parents[0] / "starry" / "img" / "spot.png"
fig, ax = plt.subplots(1)
ax.imshow(plt.imread(file))
ax.set_xticks([])
ax.set_yticks([]);

Now that we told `starry` about what the star looks like *spatially*, we need to tell it about the spectrum. There are two wavelength grids associated with a ``DopplerMap``: `wav` and `wav0`. Both of these are defined in **nanometers**.

The former, `wav`, is the wavelength grid on which the model for the observed spectral timeseries (given by the `map.flux` method) is defined. This can be accessed as `map.wav`, and can be passed in as the keyword argument `wav` when instantiating the map. We didn't explicitly provide this above, so it defaults to an array of `200` points centered at `643.0 nm`, the wavelength of an `FeI` line commonly used in Doppler imaging.

The latter, `wav0`, is the wavelength grid on which the *local, rest frame spectrum* (given by the `map.spectrum` property) is defined. This can be accessed as `map.wav0`, and can be passed in as the keyword argument `wav0` when instantiating the map. Again, we didn't explicitly provide this above, so it defaults to a similar array, but with extra padding on either side:

In [ ]:
for tick in map.wav0:
    plt.plot([tick, tick], [0.4, 0.45], "C1-", lw=0.5)
plt.annotate("wav0", xy=(0.5, 0.3), xycoords="axes fraction", ha="center", color="C1")
for tick in map.wav:
    plt.plot([tick, tick], [0.55, 0.6], "C0-", lw=0.5)
plt.annotate("wav", xy=(0.5, 0.65), xycoords="axes fraction", ha="center", color="C0")
plt.annotate(
    s="",
    xy=(map.wav[-1], 0.575),
    xycoords="data",
    xytext=(map.wav0[-1], 0.575),
    textcoords="data",
    arrowprops=dict(arrowstyle="<|-|>", lw=1, color="C1"),
)
plt.annotate(
    s="",
    xy=(map.wav0[0], 0.575),
    xycoords="data",
    xytext=(map.wav[0], 0.575),
    textcoords="data",
    arrowprops=dict(arrowstyle="<|-|>", lw=1, color="C1"),
)
plt.annotate(
    "padding",
    xy=(0.89, 0.65),
    xycoords="axes fraction",
    ha="center",
    color="C1",
    fontsize=10,
)
plt.annotate(
    "padding",
    xy=(0.11, 0.65),
    xycoords="axes fraction",
    ha="center",
    color="C1",
    fontsize=10,
)
plt.xlabel("wavelength (nm)")
plt.yticks([])
plt.ylim(0, 1);

Why the padding? And why make a distinction between these two wavelength arrays in the first place? That's because the values near the edge of the observed spectrum typically depend on a little bit of the rest frame spectrum that lies *beyond* the edge, thanks to the Doppler shift. The amount of padding is proportional to $v \sin i$: if the star is rotating very quickly, we need a lot of padding to ensure there are no edge effects in computing the model for the observed spectrum.

The user is free to provide whatever arrays they want for `wav` and `wav0`, but if `wav0` is insufficiently padded, ``starry`` will throw a warning:

In [ ]:
starry.DopplerMap(wav=np.linspace(500, 501, 100), wav0=np.linspace(500, 501, 100));

We'll discuss these wavelength grids (and how ``starry`` interpolates between them) in more detail below. For now, let's stick to the default grid for the rest frame spectrum, ``wav0``, and add a single narrow Gaussian absorption line at the central wavelength.

In [ ]:
map.load(spectra=1.0 - 0.75 * np.exp(-0.5 * (map.wav0 - 643.0) ** 2 / 0.025 ** 2))

Here's what that looks like:

In [ ]:
plt.plot(map.wav0, map.spectrum[0], "C1")
plt.xlabel("rest frame wavelength (nm)")
plt.ylabel("intensity");

We are now ready to compute the model for the observed spectrum. This is done by calling ``flux()``:

In [ ]:
theta = np.linspace(0, 360, 20)
flux = map.flux(theta=theta)

In the expression above, ``theta`` is the angular phase of the star. Note that ``theta`` must be an array of length equal to ``map.nt``, the number of epochs we told ``starry`` about earlier. The ``flux`` method returns a two-dimensional array of fluxes at each wavelength (or, alternatively, spectra at each point in time):

In [ ]:
flux.shape

In this case, that's ``20`` spectra, one at each phase ``theta``, each containing ``300`` wavelength bins. As we discussed above, the wavelength grid for the flux is given by ``map.wav``. Let's visualize our model:

In [ ]:
plt.plot(map.wav, flux.T, color="C0", lw=1, alpha=0.5)
plt.xlabel("wavelength (nm)")
plt.ylabel("flux");

Finally, we can look at just how Doppler shifted our spectrum is relative to the rest frame spectrum:

In [ ]:
plt.plot(map.wav0, map.spectrum[0], color="C1", lw=1, label="rest frame spectrum")
plt.plot(
    map.wav,
    flux[0],
    color="C0",
    lw=1,
    alpha=0.5,
    label="observed spectrum",
)
plt.plot(map.wav, flux[1:].T, color="C0", lw=1, alpha=0.5)
plt.legend()
plt.xlabel("wavelength (nm)")
plt.ylabel("normalized flux");